In [2]:
import tensorflow as tf
import numpy as np 
import pandas as pd 
from data_loader import bench, squat, deadlift

neal deadlift 0
brian deadlift 0
brian bench 0
tejas bench 0
novel bench 0
neal bench 0
brian bench 1
brian bench 2
tejas bench 1
novel bench 1
novel bench 2
brian bench 3
tejas bench 2
brian bench 4
tejas bench 3
neal bench 1
brian bench 5
novel bench 3
brian bench 6
brian squat 0
novel squat 0
brian squat 1
brian squat 2
tejas squat 0
novel squat 1
brian squat 3
novel squat 2
novel squat 3
neal squat 0
brian squat 4
{'bench': {'brian': 7, 'novel': 4, 'tejas': 4, 'neal': 2}, 'squat': {'brian': 5, 'novel': 4, 'tejas': 1, 'neal': 1}, 'deadlift': {'brian': 1, 'novel': 0, 'tejas': 0, 'neal': 1}}
                           relativeAltitude  seconds_elapsed    pressure  \
name  workout  set_number                                                  
brian bench    0                       0.00         7.011909  100.453018   
               0                       0.00         7.867835  100.453293   
               0                      -0.03        10.968242  100.454849   
               0    

In [14]:
class RepCount(tf.keras.Model):
    def __init__(self):
        super(RepCount, self).__init__()
        self.values = tf.constant([0], dtype=tf.float32)

    def heaviside_custom(self, x, values):
        ones = tf.ones_like(x)
        zeros = tf.zeros_like(x)
        return tf.where(x > 0, ones, tf.where(x < 0, zeros, values))

    def call(self, tens):
        t = self.heaviside_custom(tens, self.values)
        b = t[1:] - t[:-1]
        mask = tf.reduce_any(tf.not_equal(b, 0), axis=1)
        b = tf.boolean_mask(b, mask)
        b = tf.where(tf.equal(b, -1), tf.zeros_like(b), b)
        return tf.reduce_sum(b, axis=0)

def preprocessing(dummy_input):
    padding = np.zeros((128 - dummy_input.shape[0] % 128, dummy_input.shape[1]))
    dummy_input = np.concatenate([dummy_input, padding], axis=0)
    t = np.mean(dummy_input, axis=0) 
    dummy_input = dummy_input - t
    dummy_input_2 = np.reshape(dummy_input, (-1, 128, 3))
    dummy_input_3 = tf.constant(dummy_input_2, dtype=tf.float32)
    return dummy_input_3

model = RepCount()
dummy_input = preprocessing((bench[1][['gravityX', 'gravityY', 'gravityZ']]).dropna().to_numpy())
inp = dummy_input[0]
output = model(inp)
print("Output:", output)

Output: tf.Tensor([0. 0. 0.], shape=(3,), dtype=float32)


In [15]:
saved_model_dir = "models/model2"
tf.saved_model.save(model, saved_model_dir)


INFO:tensorflow:Assets written to: models/model2/assets


INFO:tensorflow:Assets written to: models/model2/assets


In [16]:
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

with open("rep_count_model.tflite", "wb") as f:
    f.write(tflite_model)


W0000 00:00:1741833178.329718 7876146 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1741833178.329746 7876146 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-03-12 19:32:58.329928: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: models/model2
2025-03-12 19:32:58.330084: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-03-12 19:32:58.330088: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: models/model2
2025-03-12 19:32:58.331091: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-03-12 19:32:58.334205: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: models/model2
2025-03-12 19:32:58.336382: I tensorflow/cc/saved_model/loader.cc:466] SavedModel load for tags { serve }; Status: success: OK. Took 6455 microseconds.
2025-03-12 19:32:58.349787: I tensorflow/compiler/mlir/

In [17]:
interpreter = tf.lite.Interpreter(model_path="rep_count_model.tflite")
interpreter.allocate_tensors()


input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Input details:", input_details)
print("Output details:", output_details)

input_shape = input_details[0]['shape']

Input details: [{'name': 'serving_default_inputs:0', 'index': 0, 'shape': array([128,   3], dtype=int32), 'shape_signature': array([128,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Output details: [{'name': 'PartitionedCall:0', 'index': 29, 'shape': array([3], dtype=int32), 'shape_signature': array([3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [18]:
dummy_input.shape

TensorShape([10, 128, 3])

In [19]:
interpreter.set_tensor(input_details[0]['index'], dummy_input[0])
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])
print("Model output:", output_data)


Model output: [0. 0. 0.]


In [24]:
net = np.zeros((1, 3))
for e in dummy_input:
    interpreter.set_tensor(input_details[0]['index'], e)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    print("Model output:", output_data)
    net += output_data
    
print(net)

Model output: [0. 0. 0.]
Model output: [0. 0. 0.]
Model output: [0. 1. 1.]
Model output: [0. 1. 1.]
Model output: [0. 1. 0.]
Model output: [0. 1. 1.]
Model output: [0. 1. 1.]
Model output: [0. 0. 1.]
Model output: [0. 1. 1.]
Model output: [1. 1. 0.]
[[1. 7. 6.]]


In [27]:
model(np.reshape(dummy_input, (dummy_input.shape[0]*dummy_input.shape[1], 3)))

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([1., 7., 6.], dtype=float32)>